<a href="https://colab.research.google.com/github/gviacava-code/FakeNewsDetector/blob/main/deeplearning_final_EMBEDDINGS_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import re
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
#from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
#import seaborn as sns

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.preprocessing import sequence

In [ ]:
!pip install -U sentence-transformers
# https://www.sbert.net/
from sentence_transformers import SentenceTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fake_df = pd.read_csv('/content/drive/MyDrive/TFM/Dataset/Fake.csv')
true_df = pd.read_csv('/content/drive/MyDrive/TFM/Dataset/True.csv')

In [ ]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [ ]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [ ]:
fake_df.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


In [ ]:
print(fake_df.isnull().sum())
print(true_df.isnull().sum())

title      0
text       0
subject    0
date       0
dtype: int64
title      0
text       0
subject    0
date       0
dtype: int64


In [ ]:
print(fake_df.subject.unique())
print(true_df.subject.unique())

['News' 'politics' 'Government News' 'left-news' 'US_News' 'Middle-east']
['politicsNews' 'worldnews']


In [ ]:
fake_df.drop(['subject', 'date'], axis=1, inplace=True)
true_df.drop(['subject', 'date'], axis=1, inplace=True)

In [ ]:
fake_df['class'] = 0
true_df['class'] = 1

In [ ]:
fake_df

,title,text,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0
...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,0


In [ ]:
news_df = pd.concat([fake_df, true_df], ignore_index=True, sort=False)

In [ ]:
news_df

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [ ]:
news_df.to_excel('excel_dataframe.xlsx', index=False)

In [ ]:
news_df

,title,text,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0
...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,1
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",1
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,1
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,1


In [ ]:
news_df['text'] = news_df['title'] + news_df['text']
news_df.drop(['title'], axis=1, inplace=True)

In [ ]:
news_df.iloc[1,0]

' Drunk Bragging Trump Staffer Started Russian Collusion InvestigationHouse Intelligence Committee Chairman Devin Nunes is going to have a bad day. He s been under the assumption, like many of us, that the Christopher Steele-dossier was what prompted the Russia investigation so he s been lashing out at the Department of Justice and the FBI in order to protect Trump. As it happens, the dossier is not what started the investigation, according to documents obtained by the New York Times.Former Trump campaign adviser George Papadopoulos was drunk in a wine bar when he revealed knowledge of Russian opposition research on Hillary Clinton.On top of that, Papadopoulos wasn t just a covfefe boy for Trump, as his administration has alleged. He had a much larger role, but none so damning as being a drunken fool in a wine bar. Coffee boys  don t help to arrange a New York meeting between Trump and President Abdel Fattah el-Sisi of Egypt two months before the election. It was known before that the 

In [ ]:
model_embedding = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = model_embedding.encode(news_df['text'])
embeddings

array([[-0.0186211 ,  0.02064473,  0.13922922, ..., -0.01546407,
        -0.03919584,  0.05708383],
       [-0.02492653, -0.00332258,  0.01212538, ...,  0.00739537,
         0.09397922, -0.04776388],
       [-0.04844102, -0.00240463, -0.09763719, ..., -0.04225526,
         0.00291363, -0.02996215],
       ...,
       [ 0.09235112,  0.00027989, -0.05891405, ..., -0.04590461,
         0.0412078 , -0.06245639],
       [ 0.00544583, -0.06571464, -0.01608292, ..., -0.09882528,
        -0.04701078, -0.05175719],
       [-0.05105697, -0.00999277, -0.06186692, ..., -0.13102426,
        -0.03465617,  0.03093782]], dtype=float32)

In [ ]:
X = embeddings
y = news_df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [ ]:
# Input for variable-length sequences of integers
model = Sequential()
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, batch_size=50, epochs=2, validation_data=(X_test, y_test))

Epoch 1/2
719/719 [==============================] - 6s 5ms/step - loss: 0.1980 - accuracy: 0.9229 - val_loss: 0.1281 - val_accuracy: 0.9514
Epoch 2/2
719/719 [==============================] - 3s 4ms/step - loss: 0.1234 - accuracy: 0.9520 - val_loss: 0.1120 - val_accuracy: 0.9572


In [ ]:
Y_predict = model.predict(X_test)
Y_predict

281/281 [==============================] - 1s 2ms/step


array([[2.4988813e-06],
       [9.8904884e-01],
       [9.6049261e-01],
       ...,
       [8.8813115e-04],
       [8.2746148e-04],
       [9.9954563e-01]], dtype=float32)

In [ ]:
#redondea las predicciones a 1 y 0
Y_predict = (Y_predict > 0.5).astype(int)

In [ ]:
#lo transforma todo a una sóla lista
Y_predict = [x[0] for x in Y_predict]
Y_predict

In [ ]:
results_df = pd.DataFrame({
    'Test': y_test,
    'Predicción': Y_predict
})

results_df

,Test,Predicción
12623,0,0
23808,1,1
36316,1,1
21779,0,0
25722,1,1
...,...,...
14750,0,0
18950,0,0
13006,0,0
17641,0,0


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, Y_predict)


array([[4517,  220],
       [ 164, 4079]])

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
f1_score(y_test, Y_predict, average='weighted')

0.9572513449832728

In [ ]:
def check_noticia(noticia):
  noticia=[]
  noticia.append(entrada)
  embeddings_test = model_embedding.encode(noticia)
  # Y_check_predict = model.predict(embeddings_test)
  Y_check_predict = model_test.predict(embeddings_test)
  if Y_check_predict > 0.5:
    return 'La noticia es Verdadera'
  else:
    return 'La noticia es Falsa'


In [ ]:
entrada=input('Ingresa una noticia en ingles y luego presiona "enter": ')
check_noticia(entrada)

Ingresa una noticia en ingles y luego presiona "enter": Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is DisturbingDonald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kin

'La noticia es Falsa'

# Model Save

In [ ]:
model.save('./drive/MyDrive/TFM/tfmodel')

# Model Reload

In [ ]:
model_test = load_model('./drive/MyDrive/TFM/tfmodel')

In [ ]:
!zip -r tfmodel.zip tfmodel

  adding: tfmodel/ (stored 0%)
  adding: tfmodel/saved_model.pb (deflated 85%)
  adding: tfmodel/fingerprint.pb (stored 0%)
  adding: tfmodel/keras_metadata.pb (deflated 83%)
  adding: tfmodel/assets/ (stored 0%)
  adding: tfmodel/variables/ (stored 0%)
  adding: tfmodel/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: tfmodel/variables/variables.index (deflated 57%)
